# Identifying Fraud from Enron Email

## Yong Yu

This is a report on the process of builing estimators for Fraud Detection using machine learning.

A more compact and summurized report can be found as 
<a href='https://raw.githubusercontent.com/yyforyongyu/nanodegree-machine-learning/master/final_project/documentation.html' target='_blank'>Documentation (html)</a>, or 
<a href='https://github.com/yyforyongyu/nanodegree-machine-learning/blob/master/final_project/documentation.ipynb' target='_blank'> Documentation (ipynb) </a>.

---

## Overview
In this report, there are series of investigations performed to make a robust, strong final estimator to predict a person-of-interest(poi). These include,
- an overview of the dataset.
- outlier cleaning.
- a performance comparison among different feature scaling methods, including MinMaxScaler, StandardScaler, and Normalizer.
- creating three features, "stock_salary_ratio", "poi_from_ratio", "poi_to_ratio", and evaluating them.
- a performance comparison between two different feature selection methods, SelectKBest and ExtraTreesClassifier.
- a performance comparison between including PCA and excluding PCA.
- a performance comparison between different classifiers, LinearSVC and KNeighborsClassifier.
- tuning algorithms using F1 score as evaluation metric.
- cross-validation on the final estimator.

Several helper functions are built for this project in poi_helper.py. Since this report only focuses on methodology in machine learning, we will not cover them here. For more details, report 
<a href='https://github.com/yyforyongyu/nanodegree-machine-learning/blob/master/final_project/poi_id.ipynb' target='_blank'>poi_id.ipynb</a>
has all the thoughts and steps in building these functions.

## Methodology
When finding a best combination out of groups of factors, there are usually two ways to think about it. One way would be simply find the best solution from each group, then chain all the solutions together to make the final combination. The assumption is that the best of each independent thing can be grouped to be the best of a new thing. In reality, this is rarely true since the best from one group might have a negtive effect on the best from another group. If we are to apply this method into the analysis, in short, we would need to first, find the best feature selection method, then find the best calssifier, lastly combine the feature selection and classifier to make the final estimator. However, from the report 
<a href='https://github.com/yyforyongyu/nanodegree-machine-learning/blob/master/final_project/poi_id.ipynb' target='_blank'>poi_id.ipynb</a>,
using a SelectKBest + LinearSVC could have an accuracy score of 0.94, same when using RandomizedLogisticRegression + KNeighbors, although the runtimes were different. However, when applying RandomizedLogisticRegression + LinearSVC, or SelectKBest + KNeighbors, accuracy scores became lower. This clearly indicates that, for each classifier algorithm, there is a best fit feature selection method. Simply chaining a best classifier and a feature selection seperately won't produce the best result. It becomes rather clear when all the algorithms were applied on both outlier-cleaned and full datasets. The best estimator for one dataset won't work on a different dataset.

So machine learning is really about finding a specific, nearly unique solution to a question, which brings me to think about the second way, exhaustively trying out combinations of all factors, rather than finding a best answer by groups. This assumes that a simple change in one unit can make a total difference. Unfortunately, this method also creates a problem, large time consumptions.

For this analysis, the dataset will be tested with or without PCA, with four feature scaling methods(including not using one), with two feature selection methods, and two classifiers, considering only 20 values to be tuned on each classifier, 20 values to be tuned on each feature selection, and 10 values to be tuned on PCA, a rough total number of combinations is,
$$ 2*4*2*2*20*20*10 = 128,000$$
And it is not likely we will conduct all the possibilies here at once, we will have to make a tradeoff.

For this analysis, the parameters of feature selection methods won't be tuned untill one best feature selection method is found, then the cross validation will be tuned, which brings the possible combination down dramstically before we start to tune on feature selection and cross validation.

# Summary of Dataset
A summary of findings,
- there are 146 data points with 21 features, and a total of 3066 obervations.
- there are 18 people who is an point of interest.
- 1,358 data points are missing.
- the top 3 features with most missing values are "loan_advances", "director_fees", and "restricted_stock_deferred".

In [ ]:
#!/usr/bin/python

import sys
import pickle
from tester import dump_classifier_and_data, test_classifier
sys.path.append("../tools/")
from poi_helper import *

### Load the dictionary containing the dataset
data_dict = pickle.load(open("final_project_dataset.pkl", "r") )

In [ ]:
# number of data points
len(data_dict.keys())

In [ ]:
# number of features available
len(data_dict['METTS MARK'])

In [ ]:
# available features
data_dict["METTS MARK"].keys()

In [ ]:
# people of interest
count = 0
for key, item in data_dict.iteritems():
    if item["poi"]:
        print key
        count += 1
count

In [ ]:
# create a dictionary for all missing values
missing = {}
for key, item in data_dict.iteritems():
    for elem, value in item.iteritems():
        if value == "NaN":
            if elem not in missing:
                missing[elem] = 1
            else:
                missing[elem] += 1

In [ ]:
# number of missing values
number_of_missing = 0
for key, item in missing.iteritems():
    number_of_missing += item
number_of_missing

In [ ]:
missing

# Outlier Investigation

As we already known in mini projects, there is an outlier named "TOTAL" in this dataset. We will need to remove it before any further analysis.

In [ ]:
# remove the outlier 'TOTAL'
data_dict.pop("TOTAL")

## Plots of the Outliers

To understand the outliers in this dataset, plots are created by using salary against every other feature but poi, which is used to color data points in each plot. As a starting point, all the available features will be selected and put into the model. Later in this report, some features will be removed based on their feature selection score.

In [ ]:
# create features for plots
# features_list is a list of strings, each of which is a feature name.
# The first feature must be "poi".
features_list = ['poi',
                 'salary',
                 'to_messages',
                 'deferral_payments',
                 'total_payments',
                 'exercised_stock_options',
                 'bonus',
                 'restricted_stock',
                 'shared_receipt_with_poi',
                 'restricted_stock_deferred',
                 'total_stock_value',
                 'expenses',
                 'loan_advances',
                 'from_messages',
                 'other',
                 'from_this_person_to_poi',
                 'director_fees',
                 'deferred_income',
                 'long_term_incentive',
                 'from_poi_to_this_person']

# format the dataset
data = featureFormat(data_dict, features_list)

# create a pandas dataframe
df = pd.DataFrame(data, columns = features_list)

In the following plots, blue color stands for poi, red color stands for non-poi.

In [ ]:
%matplotlib inline
from ggplot import *

# iter through all features
# x axis will always be salary
# poi is represented by colors of points
# the rest of features are put in y axis
for feature in features_list:
    if feature != "poi" and feature != "salary":
        print ggplot(aes(x = 'salary', y = feature, color = 'poi'),
               data = df) +\
        geom_point() +\
        ggtitle("salary against " + feature)

## Outlier Removal

The purpose of removing outliers is to prevent the model being misrepresented by extreme cases, which comes with an assumption that either the extreme cases rarely happen, or they don't carry engough valuable infomration to be kept in the model. This can be true for some of the features, but could be controversy for "total_payment" feature, and shouldn't be applied to "exercised_stock_options" as the top four outliers are all person of interest. On the other hand, if we are to treat top 10% of each feature as outliers, it is not hard to imagine that the final dataset will have much less than 90%. A large deduction in the original dataset will cause the model becoming weaker.

Given all these thoughts, we will start the cleaning experiment using a simple method. By fitting in a linear regression model, we will calculate the variance between the predicted values and true values, then treat features whoes predictions have top 10% variance as outliers. Based on that, we can then decide what to do with the outliers.


In [ ]:
### check on the score before outlier cleaning
features, labels = featureLabelSplit(data_dict, features_list)
buildRegression(features, labels)[1]

In [ ]:
### clean the outliers
### extract normal data points and outliers
cleaned_data, outliers = outlierCleaner(features, labels)

In [ ]:
### extract labels and features from cleaned_data
cleaned_labels, cleaned_features = targetFeatureSplit(cleaned_data)

# fit the model again and check the score
buildRegression(cleaned_features, cleaned_labels)[1]

A removal of the outliers improved the score of the linear model dramastically from 0.35 to 0.82. Although it's good to see improvement in score, it's always necessary to take a look at the removed outliers.

## Check on Outliers

In [ ]:
### change the data format from numpy array to python dictionary
outliers_dataset = personMapping(featureReformat(outliers, features_list), data_dict, features_list)

In [ ]:
### number of outliers
len(outliers_dataset)

In [ ]:
### name of outliers who is not a poi
for key, item in outliers_dataset.iteritems():
    if item['poi'] == 0.0:
        print key

## Strategy on Outlier Removal

As mentioned above, simply removing the outliers might cause an issue for later on analysis. While the imporvement in score of the linear model is surely tempting, do note that, this is not the model that we will use to conduct machine learning in this dataset. 

On the other hand, there's no surprising that most of the person of interest(13 out 18) are flagged as outliers given the background knowledge of Enron Fraud. In this case, the outliers are the targets we want to find, according to <a href='https://discussions.udacity.com/t/outlier-removal/7446' target='_blank'>this post in discussion forum</a>, we can manually decided to include or exclude the outliers or not in the training set. This strategy will be applied when processing the dataset.

Given the though above, when fitting in datasets later, there are 5% outliers cleaned on the training set.

# Preprocessing Features

## Feature Creation
To dig out more patterns from the dataset, three new features, "stock_salary_ratio", "poi_from_ratio", "poi_to_ratio", are created as following.

- stock_salary_ratio: stock_salary_ratio takes the result from total_stock_value divided by salary. This feature is useful based on the assumption that a person of interest usually has a unusual large stock value since it's under the table, while salary information could be more easily known by public, thus the ratio could give information to identify the poi. The bigger the ratio, the more likely it is a poi.
- poi_from_ratio: poi_from_ratio takes result from from_poi_to_this_person divided by from_messages. This feature assumes that if a person is a poi, he/she tends to have more contacts with another poi, therefore the ratio would be bigger. And same applie to feature poi_to_ratio.

In [ ]:
### add new features to dataset
for key, item in data_dict.iteritems():
    ### add stock_salary_ratio
    if item['salary'] != "NaN" and item['total_stock_value'] != "NaN":
        item['stock_salary_ratio'] = float(item['total_stock_value']) / item['salary']
    else:
        item['stock_salary_ratio'] = "NaN"
    
    ### add poi_from_ratio
    if item['from_messages'] != "NaN" and item['from_poi_to_this_person'] != "NaN":
        item['poi_from_ratio'] = float(item['from_poi_to_this_person']) / item['from_messages']
    else:
        item['poi_from_ratio'] = "NaN"
        
    ### add poi_to_ratio
    if item["to_messages"] != "NaN" and item["from_this_person_to_poi"] != "NaN":
        item["poi_to_ratio"] = float(item["from_this_person_to_poi"]) / item["to_messages"]
    else:
        item["poi_to_ratio"] = "NaN"

In [ ]:
### update features list
new_features_list = features_list + ["stock_salary_ratio", "poi_from_ratio", "poi_to_ratio"]

## Feature Scaling

Depending on the algorithms chosen, feature scaling may be necessary. In this report, three feature scaling methods are compared, including MinMaxScaler, StandardScaler, and Normalizer.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

In [ ]:
stdMeanReader(features, features_list)

In [ ]:
### check results from MinMaxScaler
stdMeanReader(features, features_list, MinMaxScaler())

In [ ]:
### check results from StandardScaler
stdMeanReader(features, features_list, StandardScaler())

In [ ]:
### check results from Normalizer
stdMeanReader(features, features_list, Normalizer())

In [ ]:
### create scalers
scalers = [('none', None),
           ('standardscaler', StandardScaler()),
           ('minmaxscaler', MinMaxScaler()),
           ('normalier', Normalizer())]

## Feature Selection

To get a better processing before any fitting into models, two feature selection methods for classification listed in 
<a href='http://scikit-learn.org/stable/modules/feature_selection.html' target='_blank'>sklearn documentations</a> 
are explored, which are SelectKBest and ExtraTreesClassifier.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier

feature_selections = [('k_best', SelectKBest(k = 'all')),
                     ('extra_tree', ExtraTreesClassifier(class_weight='auto', random_state=42))]

## PCA
PCA is imported to conduct dimensions deduction. Depending on the performances, the decision to include PCA or not will be made later.

In [ ]:
from sklearn.decomposition import PCA
pca = [('none', None),
       ('pca', PCA())]

# Pick and Prepare Classifiers

According to 
<a href='http://scikit-learn.org/stable/tutorial/machine_learning_map/' target='_blank'>this cheat sheet in sklearn</a>, 
there are at least four classification methods can be used,
- LinearSVC
- KNeighbors Classifier
- SVC
- Ensemble Classifers

In this report, we will check on LinearSVC and KNeighborsClassifier.

In [ ]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(class_weight='auto', penalty='l1', dual=False, random_state=42)

params_svc = {'linear_svc__C':[0.01, 0.03, 0.1, 0.3, 1, 3, 10],
              'linear_svc__tol': [1e-4, 1e-3, 1e-2, 1],
              'linear_svc__max_iter': [1e3, 1e4]}

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_neighbors = KNeighborsClassifier(weights='distance', algorithm='auto')

params_kneighbors = {'k_neighbors__n_neighbors': [1, 3, 10],
                     'k_neighbors__leaf_size': [2, 5, 10, 30, 50, 100]}

In [ ]:
### put all classifiers together
classifiers = [('linear_svc', linear_svc, params_svc),
               ('k_neighbors', k_neighbors, params_kneighbors)]

# Validation and Evaluation


## Validation
To prevent overfitting, a cross validation is needed to split the dataset into training and testing. StratifiedShuffleSplit is used across all tuning processes with a default test_size of 0.1. Depending on different steps, the n_iter paramter used for cross validation varies, as listed below,

- when finding best combination of feature selection and classification method, n_iter = 100;
- when tuning on the chosen estimator, n_iter = 1000;
- when tuning on the chosen feature selection, n_iter = 100.


## Evaluation
For evaluation, we will use accuracy score, f1 score, precision score, recall score and time consumption when deciding the best estimator. When performing grid search, f1 score is used as a scoring parameter.

# Exploring Algorithms

When runing the esimators on the dataset, there are 8 models generated seperately for scaled and non-scaled features. A comparison among the best choices is as following, listed as model number, feature selection method, classification method, accuracy score, F1 score, precision score, recall score, and time consumption.

In [ ]:
### create pipelines
pipelines = makePipelines(scalers, pca, feature_selections, classifiers)

In [ ]:
### train models on features list
model_sets, scores = trainModel(data_dict, features_list, pipelines,
                                filename = 'model_metrix.csv')

In [ ]:
### train models on new features list
model_sets_new, scores_new = trainModel(data_dict, new_features_list, pipelines,
                                        filename = 'model_metrix_new_features.csv')

check the results.

In [ ]:
pd.read_csv("model_metrix.csv").sort(['f1_score','time_used'], ascending= [0, 1])

In [ ]:
pd.read_csv("model_metrix_new_features.csv").sort(['f1_score','time_used'], ascending= [0, 1])

# Final Tuning on the Estimator

## Extract Information for Final Model

In [ ]:
### extract the pipeline
pipeline = model_sets[22][1]
tuning_score = scores[22]

In [ ]:
### get the training and testing set
features_train, features_test, labels_train, labels_test = trainTestSplit(data_dict, features_list)

### prepare the cross validation
sss = StratifiedShuffleSplit(labels_train, n_iter=100, random_state=42)

In [ ]:
### check on grid score
gridScoreReader(tuning_score)

In [ ]:
### extract algorithms
pca = pipeline.steps[1][1].transformer_list[0][1]
extra_tree = pipeline.steps[1][1].transformer_list[1][1]

## Check on New Features

In [ ]:
### check feature importances on features list
extra_tree_result = zip(features_list[1:], extra_tree.feature_importances_)
extra_tree_result.sort(key=lambda value:value[1], reverse=True)

### check feature importances
extra_tree_result

In [ ]:
### check feature importances on  new features list
### extract extra tree
new_extra_tree = model_sets_new[22][1].steps[1][1].transformer_list[1][1]

### get all the features
new_extra_tree_result = zip(new_features_list[1:], new_extra_tree.feature_importances_)
new_extra_tree_result.sort(key=lambda value:value[1], reverse=True)

### check feature importances
new_extra_tree_result

In [ ]:
### check pca explained variance ratio on features list
pca = pipeline.steps[1][1].transformer_list[0][1]
pca.explained_variance_ratio_ 

In [ ]:
### check pca explained variance ratio on new features list
pca_new = model_sets_new[22][1].steps[1][1].transformer_list[0][1]
pca_new.explained_variance_ratio_ 

## Tuning on ExtraTreeClassifier

In [ ]:
### record time
t0 = time()

### set the parameters
params_extra_tree = {"extra_tree_with_pca__extra_tree__n_estimators": [1, 3, 10, 30, 100],
                     "extra_tree_with_pca__extra_tree__max_features": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

### fit and search
estimator = GridSearchCV(pipeline, params_extra_tree, scoring='f1', cv=sss)
estimator.fit(features_train, labels_train)

### extract scores
score_extra_tree = estimator.grid_scores_

### get the best estimator
clf = estimator.best_estimator_

### check the model performance
crossValidate(data_dict, features_list, clf)

print "time used: ", time() - t0

In [ ]:
### extract algorithm
extra_tree = clf.steps[1][1].transformer_list[1][1]

### get the score
extra_tree_result = zip(features_list[1:], extra_tree.feature_importances_)
extra_tree_result.sort(key=lambda value:value[1], reverse=True)

In [ ]:
### check feature importances
extra_tree_result

In [ ]:
gridScoreReader(score_extra_tree)

## Tuning on PCA

In [ ]:
### record time
t0 = time()

### set the parameters
params_pca = {"extra_tree_with_pca__pca__n_components": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
              "extra_tree_with_pca__pca__whiten": [True, False]}

### fit and search
estimator = GridSearchCV(clf, params_pca, scoring='f1', cv=sss)
estimator.fit(features_train, labels_train)

### extract scores
score_pca = estimator.grid_scores_

### get the best estimator
clf = estimator.best_estimator_

### check the model performance
crossValidate(data_dict, features_list, clf)

print "time used: ", time() - t0

In [ ]:
### check on explained ratio
clf.steps[1][1].transformer_list[0][1].explained_variance_ratio_ 

In [ ]:
gridScoreReader(score_pca).sort(["whiten", "n_components"], ascending = [0, 1])

#Final Solution

After comparing among feature selection methods, classification methods, carefully tuning parameters for the methods, the best model turned out to be a combination of the following,

In [ ]:
### check on the parameters
clf.steps

In [ ]:
### prepare for the test
clf = clf
my_dataset = data_dict
features_list = features_list

### dump for testing
dump_classifier_and_data(clf, my_dataset, features_list)

In [ ]:
### check the score from tester
test_classifier(clf, my_dataset, features_list, folds = 1000)